In [1]:
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64
from requests import request
from pprint import pprint
import math
import json
import geocoder   #取得目前最近座標 需pip

In [22]:


app_id = 'FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF'
app_key = 'FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF'

class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }


if __name__ == '__main__':
    a = Auth(app_id, app_key)
    #
#     response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/EstimatedTimeOfArrival/City/NewTaipei?$format=JSON', headers= a.get_auth_header())
    #定點資料(非定時)
    real = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/EstimatedTimeOfArrival/City/Taipei?$format=JSON', headers= a.get_auth_header())

    

In [23]:
#Direction (integer): 去返程 : [0:'去程',1:'返程',2:'迴圈',255:'未知']
# StopName (NameType, optional): 站牌名 ,
# RouteName (NameType, optional): 路線名 ,
# Estimates (Array[Estimate], optional): 到站時間預估  ,
# StopStatus (integer, optional): 車輛狀態備註 : [0:'正常',1:'尚未發車',2:'交管不停靠',3:'末班車已過',4:'今日未營運'] ,

reals = json.loads(real.text)

inp = input(str("請輸入站牌"))


lu = []

for i in reals:
    lu_dic = {}
    if i['StopName']['Zh_tw'] == f'{inp}':
        try:
#             print(i['RouteName']['Zh_tw'], i['Direction'], i['EstimateTime'])
            lu_dic['路線'] = i['RouteName']['Zh_tw']
            lu_dic['方向'] = i['Direction']
            lu_dic['預計抵達'] = f"{i['EstimateTime'] / 60:.0f}" + '分鐘'
        except:
#             print(i['RouteName']['Zh_tw'], i['Direction'], 'NaN')
            lu_dic['路線'] = i['RouteName']['Zh_tw']
            lu_dic['方向'] = i['Direction']
            lu_dic['預計抵達'] = 'NaN'
        lu.append(lu_dic)
for i in lu:
    if i['方向'] == 0:
        i['方向'] = '去程'
    elif i['方向'] == 1:
        i['方向'] = '回程'
    else:
        i['方向'] = '未知'
for i in lu:
    print(i['路線'] + '/' + i['方向'] + '/' + i['預計抵達'])

14/去程/0分鐘
232/回程/13分鐘
14/回程/8分鐘
232/去程/6分鐘
